<img src="https://github.com/richardcerny/bricksflow/raw/rc-bricksflow2.1/docs/img/databricks_icon.png?raw=true" width=100/>
# Bricksflow example 2.

## Tables over files
You know parquet or delta, right. It is great format to store data. But for Data analyst it might be difficult to find and load these files for Blob Storage or S3. Because of that we try to keep everything in tables that are stored in Hive Metastore (=this is list of database and tables that show after opening Data icon on the left). Data analyst can just do quick analysys with SQL quickly and doesn't need to bother about loading paruqet format.


## Datalake structure
There are many ways how to create database and table names. You can see two approaches in DataSentics. They are pretty similar just the naming is used  bit differently. As the common concept used by Datalake is Bronze, Silver, Gold. Before this become common we used sometinhg similar Raw, Parsed, Cleansed.

**Key concept of Datalake created by DataSentics:**
- according to database and table name it is possible to find raw data in Storage and notebook/script that creates it and other way arround
- keep all source files
- ONE notebook/script creates only ONE table
- tables are defined in config (not hardcoded in the code)
- table schema is tight to the script/notebook
- data is written to a table (not to a file)
- ...

## Datalake structure in Bricksflow
Bricksflow uses so called Datalake bundle to maintain all tables within a config so you always now which tables are in use.
By default it requires just a table name and it resolves the path where to save data automatically.

##### Example of config.yaml
<img src="https://github.com/richardcerny/bricksflow/raw/rc-bricksflow2.1/docs/img/datalake_config.png?raw=true" width=1200/>

Non-default setting:
- If you want to adjust Storage path resolver you can find it here: ``\src\__myproject__\_config\bundles\datalakebundle.yaml`. It uses simple python find function to split the table names according "_".
- If you need exact path you can use variable `targetPath` to save a table to desired location. See commented example in config.yaml in picture above.



### 1. Common Datalake layers Bronze, Silver, Gold
Todo Image

### 2. ADAP Layers dataoneoff, dataregular, solutions
Todo Image

### Other Config variables
You might have noticed some variables which were used in the functions above like `source_csv_path` and `outputTableName`. This variables/parameters are gathered from config file that is editable only from Git/Local. It is not possible to change config within Databricks (yet).

There are some general config parameters


#### Basic parameters
Common parameters accessible anywhere and passed through @decorator

**Define param in config.yaml**
```
parameters:
  myparameter:
    myvalue: 'This is a sample string config value'
```
**Use param in code**
```
@dataFrameLoader("%myparameter.myvalue%", display=True)
def tbl_raw_trace_event(my_value_from_config: str, spark: SparkSession)
   print(my_value_from_config)
   ...
```

In [0]:
%run ../../../app/install_master_package

Library with this file name dbfs:/FileStore/jars/t__myproject__t/2021-01-13_14-36-37_nvrfpjmnea/t__myproject__t-0.1-py3-none-any.whl already installed. If you would like to reinstall this, please detach and re-attach the notebook to recreate your environment. Conflict with previous lib file: dbfs:/FileStore/jars/t__myproject__t/2021-01-13_14-36-37_nvrfpjmnea/t__myproject__t-0.1-py3-none-any.whl
Out[16]: False

In [0]:
from logging import Logger
from datalakebundle.table.TableManager import TableManager
from pyspark.sql import SparkSession
from pyspark.sql.dataframe import DataFrame
from databricksbundle.notebook.decorators import dataFrameLoader, transformation, dataFrameSaver
from datalakebundle.table.TableNames import TableNames

In [0]:
@dataFrameLoader("%datalakebundle.tables%", display=False)
def read_csv_covid_confirmed_usafacts(parameters_datalakebundle, spark: SparkSession, logger: Logger):
    source_csv_path = parameters_datalakebundle['bronze_covid.tbl_template_2_confirmed_cases']['params']['source_csv_path']
    logger.info(f"Reading CSV from source path: `{source_csv_path}`.")
    return (
        spark
            .read
            .format('csv')
            .option('header', 'true')
            .option('inferSchema', 'true')
            .load(source_csv_path)
            .limit(10) # only for test
    )

14:15:18 INFO - Reading CSV from source path: `dbfs:/databricks-datasets/COVID/USAFacts/covid_confirmed_usafacts.csv`.
{dbName: dev_bronze_covid, tableIdentifier: tbl_template_2_confirmed_cases, dbIdentifier: bronze_covid, tableName: tbl_template_2_confirmed_cases}

In [0]:
@transformation(read_csv_covid_confirmed_usafacts, display=True)
def rename_columns(df: DataFrame):
    return (
        df
            .withColumnRenamed('County Name', 'County_Name')
    )

countyFIPS,County_Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20,12/12/20,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20
0,Statewide Unallocated,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1074,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1001,Autauga County,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4,6,6,6,6,7,7,10,10,12,12,12,12,12,12,17,17,19,19,19,23,25,25,25,25,27,28,30,32,33,36,37,37,39,40,42,42,42,45,48,53,53,58,61,67,68,74,84,91,93,103,103,110,110,120,127,136,147,149,155,159,173,189,192,205,212,216,220,233,238,239,241,248,259,265,272,282,295,312,323,331,357,368,373,375,400,411,431,434,442,453,469,479,488,498,503,527,537,553,561,568,591,615,618,644,651,661,670,684,706,728,746,756,780,789,827,842,8

### Table schema
Each table defined in config must be tight with its schema. So you need to create schema of a table if you need to create it.
But don`t worry creating a schema is one of the last step you need to do while creating pipeline. Use display instead to show temporary results. Once you are happy with the result and you know exactly which columns are necessay then create a  schema.

<img src="https://github.com/richardcerny/bricksflow/raw/rc-bricksflow2.1/docs/img/table_schema.png?raw=true"/>



You can get your defined schema from config using this command example:
`schema = tableManager.getSchema('bronze_covid.tbl_template_2_confirmed_cases')`


TODO add schema generator to Bricksflow 
print(createPysparkSchema(add_parameter_from_config_df.schema))

TIP: Did you know that each function returns a dataframe. So if you need to test someting or get a schema for example, you can quickly do discovery on the dataframe produced by function. Just by calling `[function name].result`. It returns standard Spark Dataframe so you can test code before you wrap it into a function.



<img src="https://github.com/richardcerny/bricksflow/raw/rc-bricksflow2.1/docs/img/function_returns_df.png?raw=true"/>

### Example function result

In [0]:
df = rename_columns.result

In [0]:
print(df.printSchema())

root
-- countyFIPS: integer (nullable = true)
-- County_Name: string (nullable = true)
-- State: string (nullable = true)
-- stateFIPS: integer (nullable = true)
-- 1/22/20: integer (nullable = true)
-- 1/23/20: integer (nullable = true)
-- 1/24/20: integer (nullable = true)
-- 1/25/20: integer (nullable = true)
-- 1/26/20: integer (nullable = true)
-- 1/27/20: integer (nullable = true)
-- 1/28/20: integer (nullable = true)
-- 1/29/20: integer (nullable = true)
-- 1/30/20: integer (nullable = true)
-- 1/31/20: integer (nullable = true)
-- 2/1/20: integer (nullable = true)
-- 2/2/20: integer (nullable = true)
-- 2/3/20: integer (nullable = true)
-- 2/4/20: integer (nullable = true)
-- 2/5/20: integer (nullable = true)
-- 2/6/20: integer (nullable = true)
-- 2/7/20: integer (nullable = true)
-- 2/8/20: integer (nullable = true)
-- 2/9/20: integer (nullable = true)
-- 2/10/20: integer (nullable = true)
-- 2/11/20: integer (nullable = true)
-- 2/12/20: integer (nullable = true)
-- 2/13/20: integer (nullable = true)
-- 2/14/20: integer (nullable = true)
-- 2/15/20: integer (nullable = true)
-- 2/16/20: integer (nullable = true)
-- 2/17/20: integer (nullable = true)
-- 2/18/20: integer (nullable = true)
-- 2/19/20: integer (nullable = true)
-- 2/20/20: integer (nullable = true)
-- 2/21/20: integer (nullable = true)
-- 2/22/20: integer (nullable = true)
-- 2/23/20: integer (nullable = true)
-- 2/24/20: integer (nullable = true)
-- 2/25/20: integer (nullable = true)
-- 2/26/20: integer (nullable = true)
-- 2/27/20: integer (nullable = true)
-- 2/28/20: integer (nullable = true)
-- 2/29/20: integer (nullable = true)
-- 3/1/20: integer (nullable = true)
-- 3/2/20: integer (nullable = true)
-- 3/3/20: integer (nullable = true)
-- 3/4/20: integer (nullable = true)
-- 3/5/20: integer (nullable = true)
-- 3/6/20: integer (nullable = true)
-- 3/7/20: integer (nullable = true)
-- 3/8/20: integer (nullable = true)
-- 3/9/20: integer (nullable = true)
-- 3/10/20: integer (nullable = true)
-- 3/11/20: integer (nullable = true)
-- 3/12/20: integer (nullable = true)
-- 3/13/20: integer (nullable = true)
-- 3/14/20: integer (nullable = true)
-- 3/15/20: integer (nullable = true)
-- 3/16/20: integer (nullable = true)
-- 3/17/20: integer (nullable = true)
-- 3/18/20: integer (nullable = true)
-- 3/19/20: integer (nullable = true)
-- 3/20/20: integer (nullable = true)
-- 3/21/20: integer (nullable = true)
-- 3/22/20: integer (nullable = true)
-- 3/23/20: integer (nullable = true)
-- 3/24/20: integer (nullable = true)
-- 3/25/20: integer (nullable = true)
-- 3/26/20: integer (nullable = true)
-- 3/27/20: integer (nullable = true)
-- 3/28/20: integer (nullable = true)
-- 3/29/20: integer (nullable = true)
-- 3/30/20: integer (nullable = true)
-- 3/31/20: integer (nullable = true)
-- 4/1/20: integer (nullable = true)
-- 4/2/20: integer (nullable = true)
-- 4/3/20: integer (nullable = true)
-- 4/4/20: integer (nullable = true)
-- 4/5/20: integer (nullable = true)
-- 4/6/20: integer (nullable = true)
-- 4/7/20: integer (nullable = true)
-- 4/8/20: integer (nullable = true)
-- 4/9/20: integer (nullable = true)
-- 4/10/20: integer (nullable = true)
-- 4/11/20: integer (nullable = true)
-- 4/12/20: integer (nullable = true)
-- 4/13/20: integer (nullable = true)
-- 4/14/20: integer (nullable = true)
-- 4/15/20: integer (nullable = true)
-- 4/16/20: integer (nullable = true)
-- 4/17/20: integer (nullable = true)
-- 4/18/20: integer (nullable = true)
-- 4/19/20: integer (nullable = true)
-- 4/20/20: integer (nullable = true)
-- 4/21/20: integer (nullable = true)
-- 4/22/20: integer (nullable = true)
-- 4/23/20: integer (nullable = true)
-- 4/24/20: integer (nullable = true)
-- 4/25/20: integer (nullable = true)
-- 4/26/20: integer (nullable = true)
-- 4/27/20: integer (nullable = true)
-- 4/28/20: integer (nullable = true)
-- 4/29/20: integer (nullable = true)
-- 4/30/20: integer (nullable = true)
-- 5/1/20: integer (nullable = true)
-- 5/2/20: integer (nullable

#### Table Manager
You have your table and schema defined in config, so you can now do number of thing with it and Table Manager will take care of the thing underhood - saves data to right path, solves different environments.

It gives you these options which you can use:
- create -> create table (if already exists it throws an error)
- delete -> delete table
- recreate -> deleta and create a table (old data removed - also Delta log is removed => no Time travel)
- exists -> check table existence and returns True/False
- getSchema -> returns defined schema

Check examples of using these commands in cells bellow.

### a) You want create a table once and than append data use 'exists' & 'create'

In [0]:
@dataFrameSaver(rename_columns)
def save_table_ronze_covid_tbl_template_2_confirmed_cases(df: DataFrame, logger: Logger, tableNames: TableNames, tableManager: TableManager):
    outputTableName = tableNames.getByAlias('bronze_covid.tbl_template_2_confirmed_cases')
    
    # TableManager
    # get defined schema
    schema = tableManager.getSchema('bronze_covid.tbl_template_2_confirmed_cases')
    
    # Example of delete command
    #tableManager.delete('bronze_covid.tbl_template_2_confirmed_cases')

    if tableManager.exists('bronze_covid.tbl_template_2_confirmed_cases'):
        logger.info(f"Table {outputTableName} exists. Appending...")
    else:
        tableManager.create('bronze_covid.tbl_template_2_confirmed_cases')
        
    logger.info(f"Saving data to table: {outputTableName}")
    (
        df
            .select([field.name for field in schema.fields]) # Usage of schema from getSchema
            .write
            .option('partitionOverwriteMode', 'dynamic')
            .insertInto(outputTableName)
    )

14:15:25 INFO - Table dev_bronze_covid.tbl_template_2_confirmed_cases exists. Appending...
{dbName: dev_bronze_covid, tableIdentifier: tbl_template_2_confirmed_cases, dbIdentifier: bronze_covid, tableName: tbl_template_2_confirmed_cases} 
 14:15:25 INFO - Saving data to table: dev_bronze_covid.tbl_template_2_confirmed_cases
{dbName: dev_bronze_covid, tableIdentifier: tbl_template_2_confirmed_cases, dbIdentifier: bronze_covid, tableName: tbl_template_2_confirmed_cases}

### b) You want a table to be created each run from scratch - use 'recreate'

In [0]:
@dataFrameSaver(rename_columns)
def save_table_ronze_covid_tbl_template_2_confirmed_cases(df: DataFrame, logger: Logger, tableNames: TableNames, tableManager: TableManager):
    schema = tableManager.getSchema('bronze_covid.tbl_template_2_confirmed_cases')
    
    tableManager.recreate('bronze_covid.tbl_template_2_confirmed_cases')

    outputTableName = tableNames.getByAlias('bronze_covid.tbl_template_2_confirmed_cases')
    logger.info(f"Saving data to table: {outputTableName}")
    (
        df
            .select([field.name for field in schema.fields])
            .write
            .option('partitionOverwriteMode', 'dynamic')
            .insertInto(outputTableName)
    )

14:15:30 INFO - Deleting Hive table dev_bronze_covid.tbl_template_2_confirmed_cases
{dbName: dev_bronze_covid, tableIdentifier: tbl_template_2_confirmed_cases, dbIdentifier: bronze_covid, tableName: tbl_template_2_confirmed_cases} 
 14:15:31 INFO - Hive table dev_bronze_covid.tbl_template_2_confirmed_cases deleted
{dbName: dev_bronze_covid, tableIdentifier: tbl_template_2_confirmed_cases, dbIdentifier: bronze_covid, tableName: tbl_template_2_confirmed_cases} 
 14:15:31 INFO - Deleting HDFS files from /dev/bronze/covid/tbl_template_2_confirmed_cases.delta
{dbName: dev_bronze_covid, tableIdentifier: tbl_template_2_confirmed_cases, dbIdentifier: bronze_covid, tableName: tbl_template_2_confirmed_cases} 
 14:15:31 INFO - HDFS files deleted from /dev/bronze/covid/tbl_template_2_confirmed_cases.delta
{dbName: dev_bronze_covid, tableIdentifier: tbl_template_2_confirmed_cases, dbIdentifier: bronze_covid, tableName: tbl_template_2_confirmed_cases} 
 14:15:31 INFO - Recreating Hive table dev_bronze_covid.tbl_template_2_confirmed_cases (existed before)
{dbName: dev_bronze_covid, tableIdentifier: tbl_template_2_confirmed_cases, dbIdentifier: bronze_covid, tableName: tbl_template_2_confirmed_cases} 
 14:15:35 INFO - Saving data to table: dev_bronze_covid.tbl_template_2_confirmed_cases
{dbName: dev_bronze_covid, tableIdentifier: tbl_template_2_confirmed_cases, dbIdentifier: bronze_covid, tableName: tbl_template_2_confirmed_cases}